# Bonds prices import

In [1]:
import pandas as pd
pd.core.common.is_list_like = pd.api.types.is_list_like
import pandas_datareader as web
from pandas_datareader.moex import MoexReader
import numpy as np

In [2]:
bonds_payments = pd.read_csv('/Users/andrey_lukyanov/Google_Drive/Studies/Year_4/Курсач/Coding/Comparing-numerical-methods-for-term-structure-fitting/Data/bonds_payments.csv', index_col=0)
bonds_payments['Дата фактической выплаты'] = pd.to_datetime(bonds_payments['Дата фактической выплаты'])

In [3]:
bond_trade_codes = bonds_payments['Торговый код'].unique()

In [4]:
bonds_list = [MoexReader(code, start = '01.01.2016', end = '31.12.2016').read() for code in bond_trade_codes]
bonds_list = [bonds_list[i][bonds_list[i].BOARDID == 'TQOB'].drop_duplicates()['CLOSE'] for i in range(len(bonds_list))]

for i in range(len(bond_trade_codes)):
    bonds_list[i].name = bond_trade_codes[i]
    
bonds_prices = pd.concat(bonds_list, axis = 1)
bonds_prices.head()

,SU26205RMFS3,SU26207RMFS9,SU26208RMFS7,SU26209RMFS5,SU26210RMFS3,SU26211RMFS1,SU26212RMFS9,SU26214RMFS5,SU26215RMFS2,SU26216RMFS0,SU26217RMFS8,SU26218RMFS6,SU26219RMFS4,SU46011RMFS1,SU46012RMFS9,SU46018RMFS6,SU46019RMFS4,SU46020RMFS2,SU46022RMFS8,SU46023RMFS6
TRADEDATE,,,,,,,,,,,,,,,,,,,,
2016-01-04,92.3500,91.8999,94.6000,NaN,NaN,NaN,83.9998,NaN,86.7400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-05,92.3500,91.9995,94.3999,91.3000,91.3004,87.6996,84.1000,88.7500,87.1764,92.0500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77.448,NaN,NaN
2016-01-06,92.0999,91.5999,94.4000,91.2000,90.8900,87.6999,83.7494,88.6500,86.6000,91.7200,NaN,NaN,NaN,NaN,88.0,91.97,NaN,77.991,83.31,NaN
2016-01-11,90.4499,89.2999,93.2899,88.7000,89.5000,85.5007,80.8000,87.4000,84.2500,90.7900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76.990,NaN,109.2
2016-01-12,89.2510,87.8001,92.9899,87.5011,88.9649,84.1501,79.6579,86.3007,83.5001,89.8102,NaN,NaN,NaN,NaN,NaN,86.00,NaN,76.000,NaN,NaN


In [9]:
bonds_prices.to_csv('/Users/andrey_lukyanov/Google_Drive/Studies/Year_4/Курсач/Coding/Comparing-numerical-methods-for-term-structure-fitting/Data/bonds_prices.csv')

## Loading existing data

In [2]:
bonds_payments = pd.read_csv('/Users/andrey_lukyanov/Google_Drive/Studies/Year_4/Курсач/Coding/Comparing-numerical-methods-for-term-structure-fitting/Data/bonds_payments.csv', index_col=0)
bonds_payments['Дата фактической выплаты'] = pd.to_datetime(bonds_payments['Дата фактической выплаты'])

In [3]:
bonds_prices = pd.read_csv('/Users/andrey_lukyanov/Google_Drive/Studies/Year_4/Курсач/Coding/Comparing-numerical-methods-for-term-structure-fitting/Data/bonds_prices.csv', index_col='TRADEDATE', parse_dates=True)

In [7]:
irr_matrix = pd.read_csv('/Users/andrey_lukyanov/Google_Drive/Studies/Year_4/Курсач/Coding/Comparing-numerical-methods-for-term-structure-fitting/Data/bonds_irrs.csv', index_col=0)

# IRR calcultation

In [4]:
dates = bonds_prices.index

In [5]:
def npv(irr, cfs, yrs):  
    return np.sum(cfs / (1. + irr) ** yrs)

In [6]:
from scipy.optimize import fsolve

def irr(cfs, yrs, x0):
    return np.asscalar(fsolve(npv, x0=x0, args=(cfs, yrs)))

In [7]:
def maturities_irrs(date_number):
    irrs = []
    maturities = []
    payments_on_date = bonds_payments[bonds_payments['Дата фактической выплаты'] >= dates[date_number]]
    
    for ticker in bonds_payments['Торговый код'].unique():
    
        payments = payments_on_date[payments_on_date['Торговый код'] == ticker]['Сумма купона, RUB'] + payments_on_date[payments_on_date['Торговый код'] == ticker]['Погашение номинала, RUB'].values
        principal = payments_on_date[payments_on_date['Торговый код'] == ticker]['Погашение номинала, RUB'].values.sum()
        payments_dates = (payments_on_date[payments_on_date['Торговый код'] == ticker]['Дата фактической выплаты'] - dates[date_number]).apply(lambda x: x.days)/365
    
        investment = bonds_prices.iloc[date_number][ticker] / 100 * principal
        payments = np.insert(np.array(payments), 0, -investment)
        payments_dates = np.insert(np.array(payments_dates), 0, 0)
        
        if np.isnan(payments[0]) == True:
            irrs.append(np.nan)
            maturities.append(np.nan)
        else:
            bond_irr = irr(cfs=payments, yrs=payments_dates, x0=0.10)
            irrs.append(bond_irr)
            maturities.append(payments_dates[-1])
        
    return (np.array([maturities]), np.array([irrs]))   

In [8]:
irr_matrix = pd.DataFrame(np.zeros([252, 20]), index = dates, columns = bonds_payments['Торговый код'].unique())
maturity_matrix = pd.DataFrame(np.zeros([252, 20]), index = dates, columns = bonds_payments['Торговый код'].unique())

In [9]:
for i in range(len(dates)):
    maturity_matrix[dates[i]:dates[i]], irr_matrix[dates[i]:dates[i]] = maturities_irrs(i)    

In [16]:
maturity_matrix.to_csv('/Users/andrey_lukyanov/Google_Drive/Studies/Year_4/Курсач/Coding/Comparing-numerical-methods-for-term-structure-fitting/Data/bonds_maturities.csv')
irr_matrix.to_csv('/Users/andrey_lukyanov/Google_Drive/Studies/Year_4/Курсач/Coding/Comparing-numerical-methods-for-term-structure-fitting/Data/bonds_irrs.csv')

In [10]:
%%bash
say finished